# Load Dataset

## Load Interactions Dataset

In [20]:
with open("dataset/amazon-custom-books/amazon-custom-books.inter", "r", encoding="utf-8") as f:
    source_dataset = [line.split("\t") for line in f.read().split("\n")][1:-1]

with open("dataset/amazon-custom-movies-tv/amazon-custom-movies-tv.inter", "r", encoding="utf-8") as f:
    target_dataset = [line.split("\t") for line in f.read().split("\n")][1:-1]

In [21]:
source_users = list(set([row[0] for row in source_dataset]))
source_items = list(set([row[1] for row in source_dataset]))

target_users = list(set([row[0] for row in target_dataset]))
target_items = list(set([row[1] for row in target_dataset]))

In [22]:
all_users = list(set(source_users + target_users))
all_items = list(set(source_items + target_items))

n_users = len(all_users)
n_source_items = len(source_items)
n_target_items = len(target_items)
n_items = len(all_items)

In [23]:
print("source_interactions:", len(source_dataset))
print("target_interactions:", len(target_dataset))
print()
print("n_users:", n_users)
print("n_source_users:", len(source_users))
print("n_target_users:", len(target_users))
print()
print("n_source_items:", n_source_items)
print("n_target_items:", n_target_items)
print("n_items:", n_items)
print()
print("overlap_users:", len(source_users)+len(target_users)-n_users)
print("overlap_items:", n_source_items+n_target_items-n_items)

source_interactions: 104080
target_interactions: 120985

n_users: 3859
n_source_users: 2329
n_target_users: 2346

n_source_items: 4883
n_target_items: 4929
n_items: 9812

overlap_users: 816
overlap_items: 0


In [24]:
user_dict = {user: idx for idx, user in enumerate(all_users)}
source_item_dict = {item: idx for idx, item in enumerate(source_items)}
target_item_dict = {item: idx for idx, item in enumerate(target_items)}
item_dict = {item: idx for idx, item in enumerate(all_items)}

In [64]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, Dataset, Subset

# raw dataset to torch dataset with integrated user ids but separate item ids

tensor_source_users = torch.Tensor(np.array([user_dict[row[0]] for row in source_dataset]))
tensor_source_items = torch.Tensor(np.array([source_item_dict[row[1]] for row in source_dataset]))
tensor_source_ratings = torch.Tensor(np.array([float(row[2]) for row in source_dataset]))
tensor_source_clicks = torch.Tensor(np.array([1.0 if float(row[2]) >= 3.0 else 0.0 for row in source_dataset]))

tensor_target_users = torch.Tensor(np.array([user_dict[row[0]] for row in target_dataset]))
tensor_target_items = torch.Tensor(np.array([target_item_dict[row[1]] for row in target_dataset]))
tensor_target_ratings = torch.Tensor(np.array([float(row[2]) for row in target_dataset]))
tensor_target_clicks = torch.Tensor(np.array([1.0 if float(row[2]) >= 3.0 else 0.0 for row in target_dataset]))

source_dataset_torch = TensorDataset(tensor_source_users, tensor_source_items, tensor_source_ratings, tensor_source_clicks)
target_dataset_torch = TensorDataset(tensor_target_users, tensor_target_items, tensor_target_ratings, tensor_target_clicks)

## Load Item Vectors

In [26]:
with open("dataset/amazon-custom-books/amazon-custom-books.embed", "r", encoding="utf-8") as f:
    source_item_embed_file = [line.split("\t") for line in f.read().split("\n")][1:-1]

with open("dataset/amazon-custom-movies-tv/amazon-custom-movies-tv.embed", "r", encoding="utf-8") as f:
    target_item_embed_file = [line.split("\t") for line in f.read().split("\n")][1:-1]

In [27]:
source_item_embed_dict = {line[0]: list(map(float, line[1].replace("[", "").replace("]", "").split(","))) for line in source_item_embed_file}
target_item_embed_dict = {line[0]: list(map(float, line[1].replace("[", "").replace("]", "").split(","))) for line in target_item_embed_file}

In [28]:
reverse_source_item_dict = {source_item_dict[item]: item for item in source_items}
reverse_target_item_dict = {target_item_dict[item]: item for item in target_items}

source_item_feature_vectors = np.array([source_item_embed_dict[reverse_source_item_dict[idx]] for idx in range(n_source_items)])
target_item_feature_vectors = np.array([target_item_embed_dict[reverse_target_item_dict[idx]] for idx in range(n_target_items)])

# Model Engineering

In [45]:
source_train_dataset, source_test_dataset = torch.utils.data.random_split(source_dataset_torch, [0.8, 0.2])
target_train_dataset, target_test_dataset = torch.utils.data.random_split(target_dataset_torch, [0.8, 0.2])

In [46]:
import math
from typing import Literal

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

class CMF(nn.Module):
    """
    Collective Matrix Factorization for two domains sharing user embeddings.

    Assumes datasets yield (user_idx, item_idx, rating) with 0-based integer indices.
    """
    def __init__(
        self,
        n_users: int,
        n_items: int,
        embedding_dim: int = 64,
    ):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, embedding_dim)
        self.item_emb = nn.Embedding(n_items, embedding_dim)
        self.sigmoid = nn.Sigmoid()
        self.loss = nn.BCELoss()

        self._reset_parameters()

    def get_model_info(self):
        return {
            "model_type": "CMF",
            "n_users": self.user_emb.num_embeddings,
            "n_items": self.item_emb.num_embeddings,
            "embedding_dim": self.user_emb.embedding_dim,
        }

    def _reset_parameters(self):
        nn.init.normal_(self.user_emb.weight, 0, 0.1)
        nn.init.normal_(self.item_emb.weight, 0, 0.1)

    def predict(self, users: torch.LongTensor, items: torch.LongTensor):
        """
        Predict ratings for a batch given domain.
        domain: "source" or "target"
        """
        user_embedding = self.user_emb(users)
        item_embedding = self.item_emb(items)

        return self.sigmoid(torch.mul(user_embedding, item_embedding).sum(dim=-1)) # shape (batch,)

    def forward(self, users: torch.LongTensor, items: torch.LongTensor):
        return self.predict(users, items)
    
    def calculate_loss(self, users: torch.LongTensor, items: torch.LongTensor, clicks: torch.FloatTensor):
        preds = self.forward(users, items)
        loss = self.loss(preds, clicks)

        return loss

    def __str__(self):
        info = self.get_model_info()

        return "\n".join([f"{key}: {value}" for key, value in info.items()])

class CMFTrainer():
    def __init__(self, model: CMF):
        self.model = model

    def train(
        self,
        source_loader: DataLoader,
        target_loader: DataLoader,
        epochs: int = 10,
        lr: float = 1e-3,
        alpha: float = 0.2,
        weight_decay: float = 0.0,
        report_every: int = 1,
    ):
        """
        Jointly train on source and target loaders. Each loader yields (user, item, rating).
        """
        model = self.model
        self.epochs = epochs
        self.learning_rate = lr
        self.alpha = alpha
        self.weight_decay = weight_decay

        optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)

        for ep in range(1, epochs + 1):
            model.train()
            total_loss = 0.0
            n_batches = 0

            # Train on source domain
            for users, items, _, clicks in source_loader:
                users = users.long()
                items = items.long()
                clicks = clicks.float()

                loss = alpha*model.calculate_loss(users, items, clicks)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                n_batches += 1

            # Train on target domain
            for users, items, _, clicks in target_loader:
                users = users.long()
                items = items.long()
                clicks = clicks.float()

                loss = alpha*model.calculate_loss(users, items, clicks)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                n_batches += 1

            if ep % report_every == 0:
                avg_loss = total_loss / max(1, n_batches)
                print(f"Epoch {ep}/{epochs} — avg BCE: {avg_loss:.6f}")

        return model


    def evaluate(self, loader: DataLoader):
        """
        Return BCE Loss on provided loader (domain indicates which item embeddings to use).
        """
        model = self.model
        model.eval()
        total_loss = 0.0
        n = 0

        with torch.no_grad():
            for users, items, _, clicks in loader:
                users = users.long()
                items = items.long()
                clicks = clicks.float()

                loss = model.calculate_loss(users, items, clicks)

                total_loss += loss.item()
                n += 1
                
        bce = math.sqrt(total_loss / n) if n > 0 else float("nan")

        return bce

# Train Model

In [47]:
train_source_loader = DataLoader(source_train_dataset, batch_size=1024, shuffle=True)
train_target_loader = DataLoader(target_train_dataset, batch_size=1024, shuffle=True)
test_source_loader = DataLoader(source_test_dataset, batch_size=1024, shuffle=False)
test_target_loader = DataLoader(target_test_dataset, batch_size=1024, shuffle=False)

In [48]:
model = CMF(n_users, n_items, embedding_dim=64)
trainer = CMFTrainer(model)

trainer.train(train_source_loader, train_target_loader, epochs=20, lr=1e-3)

bce_target = trainer.evaluate(test_target_loader)

print("Target BCE:", bce_target)

Epoch 1/20 — avg BCE: 0.138805
Epoch 2/20 — avg BCE: 0.136236
Epoch 3/20 — avg BCE: 0.132551
Epoch 4/20 — avg BCE: 0.124444
Epoch 5/20 — avg BCE: 0.106470
Epoch 6/20 — avg BCE: 0.081095
Epoch 7/20 — avg BCE: 0.060146
Epoch 8/20 — avg BCE: 0.047614
Epoch 9/20 — avg BCE: 0.040434
Epoch 10/20 — avg BCE: 0.035814
Epoch 11/20 — avg BCE: 0.032432
Epoch 12/20 — avg BCE: 0.029550
Epoch 13/20 — avg BCE: 0.027017
Epoch 14/20 — avg BCE: 0.024614
Epoch 15/20 — avg BCE: 0.022374
Epoch 16/20 — avg BCE: 0.020271
Epoch 17/20 — avg BCE: 0.018237
Epoch 18/20 — avg BCE: 0.016377
Epoch 19/20 — avg BCE: 0.014663
Epoch 20/20 — avg BCE: 0.013056
Target BCE: 0.5059268536968287


In [62]:
test_target_loader.dataset.indices

[3739,
 33833,
 35424,
 43961,
 38331,
 39876,
 56404,
 108335,
 70641,
 9886,
 38054,
 52026,
 100040,
 117421,
 105592,
 70162,
 28662,
 68282,
 100284,
 78594,
 66415,
 5053,
 70411,
 23948,
 39828,
 92263,
 87407,
 85420,
 29234,
 68698,
 68995,
 112332,
 71036,
 64851,
 43850,
 66222,
 112216,
 112201,
 58488,
 99886,
 88342,
 51986,
 82038,
 83237,
 19607,
 58196,
 13183,
 18412,
 79565,
 114420,
 75706,
 84180,
 41095,
 18733,
 77720,
 77035,
 26411,
 78117,
 79316,
 90120,
 67266,
 55674,
 92500,
 22481,
 73267,
 14135,
 120564,
 77974,
 23213,
 35375,
 70520,
 90106,
 64379,
 10922,
 48122,
 116227,
 7824,
 10212,
 56330,
 10033,
 54075,
 119198,
 106888,
 110316,
 37215,
 60870,
 20879,
 113311,
 48579,
 72333,
 18888,
 56972,
 77175,
 90415,
 108135,
 22322,
 104678,
 52040,
 46370,
 62215,
 3849,
 115640,
 117309,
 73150,
 43496,
 118006,
 59111,
 111243,
 34455,
 72995,
 58131,
 49661,
 62515,
 12615,
 44251,
 31478,
 28233,
 64262,
 11654,
 44304,
 110723,
 21626,
 4664,


In [70]:
test_dataset = Subset(target_dataset_torch, test_target_loader.dataset.indices).dataset.tensors

In [71]:
preds = model.predict(test_dataset[0].long(), test_dataset[1].long())

In [72]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

acc = accuracy_score(test_dataset[3].numpy(), preds.detach().numpy() >= 0.5)
conf = precision_recall_fscore_support(test_dataset[3].numpy(), preds.detach().numpy() >= 0.5, average='binary')

In [73]:
print("acc:", acc)
print("prec:", conf[0])
print("recall:", conf[1])
print("f1:", conf[2])

acc: 0.9737570773236351
prec: 0.9754540686148129
recall: 0.9962410111135324
f1: 0.9857379648637358


# NCF

In [ ]:
import math
import torch

# Neural collaborative filtering (NCF) for cross-domain recommendation
# Uses precomputed item feature vectors for source / target domains.
# Expects these variables to exist in the notebook:
#   - user_dict (dict): maps user_id -> user_idx (0-based)
#   - n_users (int)
#   - source_item_feature_vectors (np.array or torch.Tensor)  # shape (n_source_items, feat_dim)
#   - target_item_feature_vectors (np.array or torch.Tensor)  # shape (n_target_items, feat_dim)
#   - source_loader, target_loader (torch.utils.data.DataLoader) or source_dataset_torch/target_dataset_torch
# If prebuilt loaders are not available, you can create them from source_dataset_torch / target_dataset_torch.

import torch.nn as nn

class NCF(nn.Module):
    """
    Neural Collaborative Filtering that concatenates a learned user embedding with provided
    item feature vectors (from source/target) and passes them through an MLP to predict rating.
    """
    def __init__(
        self,
        n_users: int,
        source_item_features,  # numpy array or torch.Tensor (n_source_items, feat_dim)
        target_item_features,  # numpy array or torch.Tensor (n_target_items, feat_dim)
        user_emb_dim: int = 64,
        mlp_layers: list = (128, 64, 32),
        dropout: float = 0.2,
        freeze_item_features: bool = True,
    ):
        super().__init__()
        # user embedding
        self.user_emb = nn.Embedding(n_users, user_emb_dim)

        # item features: convert to tensors and make Embedding-like lookup via from_pretrained
        src_feat = torch.as_tensor(source_item_features, dtype=torch.float32)
        tgt_feat = torch.as_tensor(target_item_features, dtype=torch.float32)
        self.src_feat_dim = src_feat.shape[1]
        self.tgt_feat_dim = tgt_feat.shape[1]

        self.item_feat_src = nn.Embedding.from_pretrained(src_feat, freeze=freeze_item_features)
        self.item_feat_tgt = nn.Embedding.from_pretrained(tgt_feat, freeze=freeze_item_features)

        # MLP that takes concat(user_emb, item_feat) -> rating scalar
        input_dim_src = user_emb_dim + self.src_feat_dim
        input_dim_tgt = user_emb_dim + self.tgt_feat_dim
        # For simplicity use same MLP architecture but different first layer shapes handled at forward

        # Build MLP for source domain and target domain separately if feature dims differ
        def build_mlp(input_dim):
            layers = []
            prev_dim = input_dim

            for hidden_dim in mlp_layers:
                layers.append(nn.Linear(prev_dim, hidden_dim))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(p=dropout))
                prev_dim = hidden_dim

            layers.append(nn.Linear(prev_dim, 1))  # final scalar
            return nn.Sequential(*layers)

        self.mlp_src = build_mlp(input_dim_src)
        # reuse architecture for target but with target input dim
        self.mlp_tgt = build_mlp(input_dim_tgt)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.user_emb.weight, 0, 0.1)

        # pretrained item features already set; MLP init
        for m in self.mlp_src:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

        for m in self.mlp_tgt:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, users: torch.LongTensor, items: torch.LongTensor, domain: str):
        """
        users: (batch,)
        items: (batch,) -- item indices local to domain (0..n_items_domain-1)
        domain: "source" or "target"
        returns: (batch,) predicted scalar ratings
        """
        u = self.user_emb(users)  # (batch, user_emb_dim)

        if domain == "source":
            i_feat = self.item_feat_src(items)  # (batch, feat_dim)
            x = torch.cat([u, i_feat], dim=-1)
            out = self.mlp_src(x).squeeze(-1)

        elif domain == "target":
            i_feat = self.item_feat_tgt(items)
            x = torch.cat([u, i_feat], dim=-1)
            out = self.mlp_tgt(x).squeeze(-1)

        else:
            raise ValueError("domain must be 'source' or 'target'")
        
        return out

# Training / evaluation helpers
def train_ncf(
    model: nn.Module,
    source_loader,
    target_loader,
    epochs: int = 10,
    lr: float = 1e-3,
    weight_decay: float = 0.0,
    device: torch.device = None,
    report_every: int = 1,
):
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = nn.MSELoss()

    for ep in range(1, epochs + 1):
        model.train()
        total_loss = 0.0
        n_batches = 0

        for users, items, ratings in source_loader:
            users = users.long().to(device)
            items = items.long().to(device)
            ratings = ratings.float().to(device)

            preds = model(users, items, domain="source")
            loss = loss_fn(preds, ratings)
            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            n_batches += 1

        for users, items, ratings in target_loader:
            users = users.long().to(device)
            items = items.long().to(device)
            ratings = ratings.float().to(device)

            preds = model(users, items, domain="target")
            loss = loss_fn(preds, ratings)
            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            n_batches += 1

        if ep % report_every == 0:
            print(f"Epoch {ep}/{epochs} avg MSE: {total_loss / max(1, n_batches):.6f}")

    return model

def evaluate_ncf(model: nn.Module, loader, domain: str, device: torch.device = None):
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    model.to(device)
    model.eval()
    se = 0.0
    n = 0
    with torch.no_grad():
        for users, items, ratings in loader:
            users = users.long().to(device)
            items = items.long().to(device)
            ratings = ratings.float().to(device)
            preds = model(users, items, domain=domain)
            se += ((preds - ratings) ** 2).sum().item()
            n += ratings.numel()
    rmse = math.sqrt(se / n) if n > 0 else float("nan")
    return rmse

In [ ]:
# ...existing code...
class NCF(nn.Module):
    """
    Neural Collaborative Filtering that concatenates a learned user embedding with provided
    item feature vectors (from source/target) and passes them through an MLP to predict rating.

    This version shares a base user embedding across domains and applies small domain-specific
    adapters (learned transforms) so the model can learn domain-specific user behavior while
    still sharing signal across domains (helpful for overlapping users).
    """
    def __init__(
        self,
        n_users: int,
        source_item_features,  # numpy array or torch.Tensor (n_source_items, feat_dim)
        target_item_features,  # numpy array or torch.Tensor (n_target_items, feat_dim)
        user_emb_dim: int = 64,
        mlp_layers: list = (128, 64, 32),
        dropout: float = 0.2,
        freeze_item_features: bool = True,
        adapter_hidden: int = None,  # if set, use a small hidden layer in adapters
    ):
        super().__init__()
        # shared base user embedding (size = total unique users across both domains)
        self.user_emb = nn.Embedding(n_users, user_emb_dim)

        # lightweight domain adapters: map base user embedding -> domain-specific user embedding
        if adapter_hidden is None:
            # single linear adapter
            self.user_adapter_src = nn.Linear(user_emb_dim, user_emb_dim)
            self.user_adapter_tgt = nn.Linear(user_emb_dim, user_emb_dim)
        else:
            self.user_adapter_src = nn.Sequential(
                nn.Linear(user_emb_dim, adapter_hidden),
                nn.ReLU(),
                nn.Linear(adapter_hidden, user_emb_dim),
            )
            self.user_adapter_tgt = nn.Sequential(
                nn.Linear(user_emb_dim, adapter_hidden),
                nn.ReLU(),
                nn.Linear(adapter_hidden, user_emb_dim),
            )

        # item features: convert to tensors and make Embedding-like lookup via from_pretrained
        src_feat = torch.as_tensor(source_item_features, dtype=torch.float32)
        tgt_feat = torch.as_tensor(target_item_features, dtype=torch.float32)
        self.src_feat_dim = src_feat.shape[1]
        self.tgt_feat_dim = tgt_feat.shape[1]

        self.item_feat_src = nn.Embedding.from_pretrained(src_feat, freeze=freeze_item_features)
        self.item_feat_tgt = nn.Embedding.from_pretrained(tgt_feat, freeze=freeze_item_features)

        # MLPs for each domain
        input_dim_src = user_emb_dim + self.src_feat_dim
        input_dim_tgt = user_emb_dim + self.tgt_feat_dim

        def build_mlp(input_dim):
            layers = []
            prev = input_dim
            for h in mlp_layers:
                layers.append(nn.Linear(prev, h))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(p=dropout))
                prev = h
            layers.append(nn.Linear(prev, 1))
            return nn.Sequential(*layers)

        self.mlp_src = build_mlp(input_dim_src)
        self.mlp_tgt = build_mlp(input_dim_tgt)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.user_emb.weight, 0, 0.1)
        # adapter init
        def init_adapter(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        self.user_adapter_src.apply(init_adapter)
        self.user_adapter_tgt.apply(init_adapter)
        # MLP init
        for m in self.mlp_src:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)
        for m in self.mlp_tgt:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, users: torch.LongTensor, items: torch.LongTensor, domain: str):
        """
        users: (batch,)
        items: (batch,) -- item indices local to domain (0..n_items_domain-1)
        domain: "source" or "target"
        returns: (batch,) predicted scalar ratings
        """
        u = self.user_emb(users)  # (batch, user_emb_dim)
        # domain-specific adapter: lets the model learn per-domain user behavior while sharing base info
        if domain == "source":
            u_dom = self.user_adapter_src(u)
            i_feat = self.item_feat_src(items)
            x = torch.cat([u_dom, i_feat], dim=-1)
            out = self.mlp_src(x).squeeze(-1)
        elif domain == "target":
            u_dom = self.user_adapter_tgt(u)
            i_feat = self.item_feat_tgt(items)
            x = torch.cat([u_dom, i_feat], dim=-1)
            out = self.mlp_tgt(x).squeeze(-1)
        else:
            raise ValueError("domain must be 'source' or 'target'")
        return out
# ...existing code...

In [ ]:
# Example usage (assumes the named variables exist in the notebook):
# - source_item_feature_vectors, target_item_feature_vectors: arrays/tensors of item features
# - source_loader, target_loader: DataLoader objects producing (user_idx, item_idx, rating)


In [ ]:
model = NCF(n_users=n_users,
                 source_item_features=source_item_feature_vectors,
                 target_item_features=target_item_feature_vectors,
                 user_emb_dim=64,
                 mlp_layers=(128,64,32),
                 dropout=0.2,
                 freeze_item_features=True)

train_ncf(model, source_loader, target_loader, epochs=50, lr=1e-3)
print("Target RMSE:", evaluate_ncf(model, target_loader, domain="target"))

Epoch 1/50 avg MSE: 3.040604
Epoch 2/50 avg MSE: 1.408292
Epoch 3/50 avg MSE: 1.271468
Epoch 4/50 avg MSE: 1.212633
Epoch 5/50 avg MSE: 1.175297
Epoch 6/50 avg MSE: 1.136314
Epoch 7/50 avg MSE: 1.113781
Epoch 8/50 avg MSE: 1.085496
Epoch 9/50 avg MSE: 1.059976
Epoch 10/50 avg MSE: 1.043349
Epoch 11/50 avg MSE: 1.024997
Epoch 12/50 avg MSE: 1.004933
Epoch 13/50 avg MSE: 0.988168
Epoch 14/50 avg MSE: 0.970328
Epoch 15/50 avg MSE: 0.953801
Epoch 16/50 avg MSE: 0.931623
Epoch 17/50 avg MSE: 0.916105
Epoch 18/50 avg MSE: 0.898908
Epoch 19/50 avg MSE: 0.881925
Epoch 20/50 avg MSE: 0.862051
Epoch 21/50 avg MSE: 0.847788
Epoch 22/50 avg MSE: 0.831449
Epoch 23/50 avg MSE: 0.812696
Epoch 24/50 avg MSE: 0.797843
Epoch 25/50 avg MSE: 0.781795
Epoch 26/50 avg MSE: 0.762961
Epoch 27/50 avg MSE: 0.748274
Epoch 28/50 avg MSE: 0.734222
Epoch 29/50 avg MSE: 0.716033
Epoch 30/50 avg MSE: 0.698651
Epoch 31/50 avg MSE: 0.688903
Epoch 32/50 avg MSE: 0.675081
Epoch 33/50 avg MSE: 0.662427
Epoch 34/50 avg MSE

In [25]:
preds = model.forward(tensor_target_users.long(), tensor_target_items.long(), 'target')

In [26]:
bin_preds = [int(e >= 3) for e in preds]
bin_ratings = [int(e >= 3) for e in tensor_target_ratings]

In [27]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

acc = accuracy_score(bin_ratings, bin_preds)
conf = precision_recall_fscore_support(bin_ratings, bin_preds, average='binary')

In [28]:
print("acc:", acc)
print("prec:", conf[0])
print("recall:", conf[1])
print("f1:", conf[2])

acc: 0.9506054469562343
prec: 0.9757467799397095
recall: 0.969846371758553
f1: 0.9727876287533127
